In [17]:
# Skapar modell för hela description text
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL.json')
description_text = data['description_text']

# Step 2: Preprocess the text data
processed_data = [simple_preprocess(text) for text in description_text]

# Step 3: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 4: Save the trained model
model.save('word2vec_model_desc_text')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.


# Börja här

In [1]:
# skapar modell för enstaka meningar i description text
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL.json')
description_text = data['description_text']

# Step 2: Tokenize sentences in the text data
sentences = []
for text in description_text:
    text_sentences = sent_tokenize(text)
    sentences.extend(text_sentences)

# Step 3: Preprocess the text data
processed_data = [simple_preprocess(sentence) for sentence in sentences]

# Step 4: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 5: Save the trained model
model.save('word2vec_model_desc_sentence')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Skriver resultatet till textfil för granskning (#Hoppa över)

In [ ]:
with open('sentences.txt', 'w', encoding='utf-8') as file:
    for sentence in sentences:
        file.write(sentence + '\n')

# Hoppa över

In [43]:
# save sentences to dataframe csv 
import pandas as pd

# Create a list of dictionaries
data_rows = []
for sentence in sentences:
    data_rows.append({'sentence': sentence})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_rows)

# Save the DataFrame to a file
df.to_csv('sentences.csv', index=False)


# Hoppa över

In [2]:
import pandas as pd

# Read the data from the file into a DataFrame
df = pd.read_csv('sentences.csv')

# Filter rows containing the specified words
filtered_df = df[df['sentence'].str.contains('analytisk', case=False)]

# Print the filtered DataFrame
print(filtered_df)


                                                sentence
18     Sista ansökningsdag är dock 2016-12-15 Kvalifi...
34     Vill du vara med och rädda liv?Vill du vara me...
93     * Har ett brinnande intresse för teknik och te...
110    Som person är du kreativ och engagerad med god...
125    * Göra ändringar i texter, bilder och banners ...
...                                                  ...
67329  Du bör vara kreativ, analytisk och ha en stark...
68113  Du har en analytisk förmåga samtidigt som du k...
68347  För att trivas i rollen som konsult hos oss tr...
68492  Vi tror att du är analytiskt lagd, har en nyfi...
68585  Du är analytiskt lagd och inte rädd för att ko...

[415 rows x 1 columns]


# Hoppa över

In [46]:
# sparar alla meningar till nytt dataset
df.to_csv('sentences__filtered_analys.csv', index=False)

# Cosine similarity på enstaka ord

In [4]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "analytisk"

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

# Find similar sentences
similar_sentences = model.wv.most_similar(positive=processed_target, topn=10)

# Print the similar sentences
for sentence, similarity in similar_sentences:
    print(f"Similarity: {similarity:.2f} Sentence: {sentence}")


Similarity: 0.91 Sentence: pedagogisk
Similarity: 0.91 Sentence: noggrann
Similarity: 0.89 Sentence: strukturerad
Similarity: 0.89 Sentence: kommunikativ
Similarity: 0.87 Sentence: ansvarstagande
Similarity: 0.87 Sentence: serviceinriktad
Similarity: 0.85 Sentence: serviceminded
Similarity: 0.85 Sentence: problemlösande
Similarity: 0.85 Sentence: självgående
Similarity: 0.84 Sentence: social


# Söker igenom alla meningar och utan ord prio

In [3]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    if len(sentence) > 0:
        similarity = model.wv.n_similarity(processed_target, sentence)
        similar_sentences.append((sentence, similarity))

# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:10]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


['du', 'är', 'diplomatisk', 'har', 'god', 'analytisk', 'förmåga', 'och', 'kan', 'enkelt', 'röra', 'dig', 'mellan', 'detaljer', 'och', 'helheter']
Similarity: 1.00 Sentence: du är diplomatisk har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter
Similarity: 0.94 Sentence: du är bra på problemlösning och har god analytisk förmåga
Similarity: 0.94 Sentence: god kommunikativ och social förmåga samt ordning och reda är starka kort hos dig
Similarity: 0.94 Sentence: du är analytisk och har en god förmåga att prioritera
Similarity: 0.93 Sentence: du är serviceinriktad noggrann ansvarskännande och har god förmåga att kommunicera med din omgivning
Similarity: 0.93 Sentence: du är strukturerad självgående och har förmåga att prioritera
Similarity: 0.93 Sentence: är du drivande och strukturerad och har förmåga att samarbeta
Similarity: 0.93 Sentence: du är driven och ansvarstagande och har god kommunikativ förmåga
Similarity: 0.93 Sentence: du har dessutom en god kommunik

# PRIORITERAR ETT ORD FÖR VEKTORN SÅ ALLA MENINGAR UTAN ORDET EXKLUDERAS FRÅN SIMILARITETEN

In [7]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    if len(sentence) > 0 and 'analytisk' in sentence:  # Check if 'priority_word' is present in the sentence
        similarity = model.wv.n_similarity(processed_target, sentence)
        similar_sentences.append((sentence, similarity))

# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:10]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


['du', 'är', 'diplomatisk', 'har', 'god', 'analytisk', 'förmåga', 'och', 'kan', 'enkelt', 'röra', 'dig', 'mellan', 'detaljer', 'och', 'helheter']
Similarity: 1.00 Sentence: du är diplomatisk har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter
Similarity: 0.94 Sentence: du är bra på problemlösning och har god analytisk förmåga
Similarity: 0.94 Sentence: du är analytisk och har en god förmåga att prioritera
Similarity: 0.92 Sentence: vad vi vill att du har med dig har god analytisk kapacitet till helhetstänk har god förmåga att uttrycka dig väl tal och skrift har en god social kompetens har bra förmåga att arbeta såväl grupp som självständigt sätter kunden fokus och ge dem god service är serviceinriktad stresstålig noggrann strukturerad och ansvarsfull meriterande är om du har tidigare erfarenhet av oas äga ik collekt melos softronic är ett företag med såväl bredd som djup
Similarity: 0.92 Sentence: som person är du strukturerad har god analytisk förmåga och fö

In [ ]:
pro

In [8]:
from gensim import corpora, models

# Preprocess the sentences
#processed_data = [...]  # Your preprocessed sentences

# Create a dictionary from the preprocessed data
dictionary = corpora.Dictionary(processed_data)

# Create a corpus (bag of words representation) using the dictionary
corpus = [dictionary.doc2bow(sentence) for sentence in processed_data]

# Train the LDA model
num_topics = 5  # Specify the number of topics
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print the topics and their top keywords
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

# Get the topic distribution for each sentence in the dataset
topic_distributions = []
for sentence in processed_data:
    bow = dictionary.doc2bow(sentence)
    topic_distribution = lda_model.get_document_topics(bow)
    topic_distributions.append(topic_distribution)

# Visualize the dominant topic for each sentence
for i, topic_dist in enumerate(topic_distributions):
    dominant_topic = max(topic_dist, key=lambda x: x[1])
    print(f"Sentence {i+1}: Dominant Topic - {dominant_topic[0]}")


(0, '0.012*"and" + 0.012*"the" + 0.009*"source" + 0.008*"open" + 0.008*"react"')
(1, '0.041*"och" + 0.030*"är" + 0.026*"på" + 0.026*"om" + 0.022*"mer"')
(2, '0.043*"vi" + 0.032*"du" + 0.028*"att" + 0.023*"din" + 0.022*"är"')
(3, '0.067*"och" + 0.052*"att" + 0.041*"du" + 0.027*"med" + 0.027*"som"')
(4, '0.069*"och" + 0.036*"av" + 0.030*"har" + 0.029*"inom" + 0.022*"vi"')
Sentence 1: Dominant Topic - 1
Sentence 2: Dominant Topic - 3
Sentence 3: Dominant Topic - 3
Sentence 4: Dominant Topic - 3
Sentence 5: Dominant Topic - 4
Sentence 6: Dominant Topic - 3
Sentence 7: Dominant Topic - 3
Sentence 8: Dominant Topic - 3
Sentence 9: Dominant Topic - 3
Sentence 10: Dominant Topic - 3
Sentence 11: Dominant Topic - 3
Sentence 12: Dominant Topic - 3
Sentence 13: Dominant Topic - 3
Sentence 14: Dominant Topic - 3
Sentence 15: Dominant Topic - 3
Sentence 16: Dominant Topic - 3
Sentence 17: Dominant Topic - 4
Sentence 18: Dominant Topic - 2
Sentence 19: Dominant Topic - 4
Sentence 20: Dominant Topic 

# Genererar mening utifrån best guess på efterkommande ord



In [11]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

# Initialize an empty list to store the generated sentence
generated_sentence = []

# Iterate over each word in the target sentence
for target_word in processed_target:
    # Check if the target word is in the vocabulary of the Word2Vec model
    if target_word in model.wv:
        # Get the most similar word to the target word
        similar_word = model.wv.most_similar(positive=[target_word], topn=1)[0][0]
        generated_sentence.append(similar_word)

# Print the generated sentence
generated_sentence = ' '.join(generated_sentence)
print(f"Generated Sentence: {generated_sentence}")


Generated Sentence: man vara asd ha hög pedagogisk förmågan samt måste ladda befinna sig utgångspunkten helheten samt helheten
